In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
# get dataframes
df_pid = pd.read_csv('./hospiedata/data/cleaned/id_to_attrs_no_multi_ehr.csv', parse_dates=[''])
df_care = pd.read_csv('./hospiedata/data/care_translated_all_cols.csv', parse_dates=['TimeStamp_Insert'])
df_ht = pd.read_csv('./hospiedata/data/cleaned/therapy_cleaned.csv')

C:\Users\prate\Anaconda3\envs\det2\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,11,13,15,16,17,18,19,20,22,23,24,25,26,27,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\prate\Anaconda3\envs\det2\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# fix dtype errors
df_pid.dtypes

idehr                       int64
Age                       float64
Sex                        object
when_needed               float64
standard                  float64
StartOfHospitalization     object
EndOfHospitalization       object
Diagnosis                  object
EHR                       float64
AMB                       float64
death_time                 object
duration_in_hospital       object
survival_time              object
survival_time_days        float64
days_in_hospital          float64
age_category               object
dtype: object